# Libraries

In [1]:
# %pip install -U transformers
# %pip install -U datasets
# %pip install -U accelerate
# %pip install -U peft
# %pip install -U trl
# %pip install -U bitsandbytes

In [2]:
import os, torch, wandb

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)

from datasets import load_dataset, concatenate_datasets
from trl import SFTTrainer, setup_chat_format
from dataclasses import dataclass

C:\Users\USER_ELISEY\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setup Huggingface 🤗 & Wandb

In [5]:
@dataclass
class Config:
#     model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
#     model_name = "AnatoliiPotapov/T-lite-instruct-0.1"
    model_name = "notused"
    dataset_name = "notused"
    new_model = "russia_chad"
    torch_dtype = torch.float16
    attn_implementation = "eager"
cfg = Config()

# Loading model and tokenizer

In [6]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=cfg.torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    "C:\\Users\\USER_ELISEY\\gemma",
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=cfg.attn_implementation
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.03s/it]


In [7]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("C:\\Users\\USER_ELISEY\\gemma")
tokenizer.padding_side = 'right'
tokenizer.padding_token = '<|pad|>'
print(len(tokenizer))

256000


## LoRA adapter

In [8]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

# Data

## Load

In [9]:
dataset = load_dataset('miracl/miracl', 'ru', trust_remote_code=True)
# data_eval["dev"]['negative_passages']

In [10]:
dataset

DatasetDict({
    dev: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages'],
        num_rows: 1252
    })
    testB: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages'],
        num_rows: 718
    })
    train: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages'],
        num_rows: 4683
    })
    testA: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages'],
        num_rows: 911
    })
})

## Format to chat 

In [11]:
def format_chat_template(row):
    row_json = [{"role": "user", "content": row["query"]},
               {"role": "assistant", "content": row["positive_passages"][0]["text"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

In [12]:
dataset = concatenate_datasets([
    dataset['dev'],
    dataset['train']
])

dataset = dataset.remove_columns('negative_passages')

In [13]:
dataset['positive_passages'][0][0]["text"]

'Кари́бский кризис\xa0— исторический термин, определяющий чрезвычайно напряжённое политическое, дипломатическое и военное противостояние между Советским Союзом и Соединёнными Штатами в октябре 1962 года, которое было вызвано размещением США ядерного оружия в Турции в 1961 году и впоследствии тайной переброской и размещением на Кубе военных частей и подразделений Вооружённых Сил СССР, техники и вооружения, включая ядерное оружие. Кризис мог привести к глобальной ядерной войне. Кубинцы называют его «Октябрьским кризисом» (), в США распространено название «Кубинский ракетный кризис» ().'

In [14]:
dataset = dataset.map(
    format_chat_template,
    num_proc=1,
)

## Select only part

In [15]:
dataset_sh = dataset.shuffle(seed=2024)#.select(range(10_000))
dataset_sh

Dataset({
    features: ['query_id', 'query', 'positive_passages', 'text'],
    num_rows: 5935
})

In [16]:
dataset_sh = dataset_sh.train_test_split(0.1)

In [17]:
dataset_sh

DatasetDict({
    train: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'text'],
        num_rows: 5341
    })
    test: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'text'],
        num_rows: 594
    })
})

# Train model

## Training arguments

In [18]:
training_arguments = TrainingArguments(
    output_dir=cfg.new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
#     num_train_epochs=1,
    max_steps=500,
    eval_strategy="steps",
    eval_steps=100,
    logging_steps=10,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=True,
    bf16=False,
    group_by_length=True,
    report_to="wandb",
    run_name="Llama-3.1-medicine",
)

In [19]:
# print(len(tokenizer))

In [20]:
# model.resize_token_embeddings(256000)

## Train model

In [21]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_sh["train"],
    eval_dataset=dataset_sh["test"],
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

C:\Users\USER_ELISEY\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
C:\Users\USER_ELISEY\AppData\Roaming\Python\Python311\site-packages\trl\trainer\sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
C:\Users\USER_ELISEY\AppData\Roaming\Python\Python311\site-packages\trl\trainer\sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
Map: 100%|██████████| 594/594 [00:00<00:00, 6795.04 examples/s]
max_steps is given, it will o

In [ ]:
trainer.train()

  2%|▏         | 10/500 [08:31<5:48:18, 42.65s/it]

{'loss': 2.3, 'grad_norm': 2.1840381622314453, 'learning_rate': 0.0002, 'epoch': 0.0}


  4%|▍         | 20/500 [13:25<3:47:05, 28.39s/it]

{'loss': 2.0288, 'grad_norm': 1.8392691612243652, 'learning_rate': 0.0001959183673469388, 'epoch': 0.01}


  6%|▌         | 30/500 [18:52<4:17:19, 32.85s/it]

...

{'loss': 1.4698, 'grad_norm': 2.0734493732452393, 'learning_rate': 4.489795918367347e-06, 'epoch': 0.18}


100%|██████████| 500/500 [9:53:19<00:00, 15.90s/it]

{'loss': 1.4128, 'grad_norm': 2.824575901031494, 'learning_rate': 4.081632653061225e-07, 'epoch': 0.19}


                                                   
100%|██████████| 500/500 [10:55:23<00:00, 15.90s/it]

{'eval_loss': 1.570472002029419, 'eval_runtime': 3723.8734, 'eval_samples_per_second': 0.16, 'eval_steps_per_second': 0.16, 'epoch': 0.19}


In [ ]:
path_to_save = "Llama-finetuned"
trainer.save_model(path_to_save)
model.save_pretrained(path_to_save)
tokenizer.save_pretrained(path_to_save)

In [ ]:
del model, tokenizer, trainer

# Compare models

## Init casual LLM

In [ ]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=cfg.torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
casual_model = AutoModelForCausalLM.from_pretrained(
    cfg.model_name,
    quantization_config=bnb_config,
#     device_map="auto",
    attn_implementation=cfg.attn_implementation
)

tokenizer = tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)
tokenizer.padding_side = 'right'
tokenizer.padding_token = '<|pad_token|>'

In [ ]:
casual_model, tokenizer = setup_chat_format(casual_model, tokenizer)

## Get answers

In [ ]:
def generate_answer(model, prompt):
    chat = [
        { "role": "user", "content": prompt },
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=150)

    return(tokenizer.decode(outputs[0]))

# Comprasion

In [ ]:
q1 = "I have severe headaches help me please"
q2 = "I have a suspiciously large mole. Could I have cancer? How can I determine this at home?"
q3 = "What does abutment of the nerve root mean?"

In [ ]:
generate_answer(model, q1)

In [ ]:
generate_answer(model, q2)

In [ ]:
generate_answer(model, q3)

In [ ]:
# Free gpu memory
import numba
numba.cuda.close()

In [ ]:
print(generate_answer(casual_model, q1))

In [ ]:
generate_answer(casual_model, q2)

In [ ]:
generate_answer(casual_model, q3)